***
***
# 使用Graphlab实现电影推荐的隐语义模型
> ## （Latent Factor Model， LFM）
***
***



In [9]:
import graphlab
graphlab.canvas.set_target("ipynb")
rating_sf = graphlab.SFrame('ratings')
users = graphlab.SFrame('users')
items = graphlab.SFrame('items')

In [3]:
rating_sf

Columns:
	user_id	int
	movie_id	int
	rating	int
	timestamp	int

Rows: 1000209

Data:
+---------+----------+--------+-----------+
| user_id | movie_id | rating | timestamp |
+---------+----------+--------+-----------+
|    1    |   1193   |   5    | 978300760 |
|    1    |   661    |   3    | 978302109 |
|    1    |   914    |   3    | 978301968 |
|    1    |   3408   |   4    | 978300275 |
|    1    |   2355   |   5    | 978824291 |
|    1    |   1197   |   3    | 978302268 |
|    1    |   1287   |   5    | 978302039 |
|    1    |   2804   |   5    | 978300719 |
|    1    |   594    |   4    | 978302268 |
|    1    |   919    |   4    | 978301368 |
+---------+----------+--------+-----------+
[1000209 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [10]:
users

Columns:
	user_id	int
	gender	str
	age	str
	occupation	str
	zip-code	str

Rows: 6040

Data:
+---------+--------+-----+------------+----------+
| user_id | gender | age | occupation | zip-code |
+---------+--------+-----+------------+----------+
|    1    |   F    |  1  |     10     |  48067   |
|    2    |   M    |  56 |     16     |  70072   |
|    3    |   M    |  25 |     15     |  55117   |
|    4    |   M    |  45 |     7      |  02460   |
|    5    |   M    |  25 |     20     |  55455   |
|    6    |   F    |  50 |     9      |  55117   |
|    7    |   M    |  35 |     1      |  06810   |
|    8    |   M    |  25 |     12     |  11413   |
|    9    |   M    |  25 |     17     |  61614   |
|    10   |   F    |  35 |     1      |  95370   |
+---------+--------+-----+------------+----------+
[6040 rows x 5 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [11]:
items

Columns:
	movie_id	int
	title	str
	genre	str

Rows: 3883

Data:
+----------+-------------------------------+------------------------------+
| movie_id |             title             |            genre             |
+----------+-------------------------------+------------------------------+
|    1     |        Toy Story (1995)       | Animation|Children's|Comedy  |
|    2     |         Jumanji (1995)        | Adventure|Children's|Fantasy |
|    3     |    Grumpier Old Men (1995)    |        Comedy|Romance        |
|    4     |    Waiting to Exhale (1995)   |         Comedy|Drama         |
|    5     | Father of the Bride Part I... |            Comedy            |
|    6     |          Heat (1995)          |    Action|Crime|Thriller     |
|    7     |         Sabrina (1995)        |        Comedy|Romance        |
|    8     |      Tom and Huck (1995)      |     Adventure|Children's     |
|    9     |      Sudden Death (1995)      |            Action            |
|    10    |        GoldenEye (1995)       |  Action|Adventure|Thriller   |
+----------+-------------------------------+------------------------------+
[3883 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [4]:
dir(graphlab.recommender)

['__all__',
 '__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 '__path__',
 'create',
 'factorization_recommender',
 'item_content_recommender',
 'item_similarity_recommender',
 'popularity_recommender',
 'ranking_factorization_recommender',
 'util']

In [16]:
item_data = graphlab.SFrame({"my_item_id" : range(4),
                                 "data_1" : [ [1, 0], [1, 0], [0, 1], [0.5, 0.5] ],
                                 "data_2" : [ [0, 1], [1, 0], [0, 1], [0.5, 0.5] ] })
item_data

Columns:
	data_1	array
	data_2	array
	my_item_id	int

Rows: 4

Data:
+------------+------------+------------+
|   data_1   |   data_2   | my_item_id |
+------------+------------+------------+
| [1.0, 0.0] | [0.0, 1.0] |     0      |
| [1.0, 0.0] | [1.0, 0.0] |     1      |
| [0.0, 1.0] | [0.0, 1.0] |     2      |
| [0.5, 0.5] | [0.5, 0.5] |     3      |
+------------+------------+------------+
[4 rows x 3 columns]

In [18]:
m = graphlab.recommender.item_content_recommender.create(item_data, "my_item_id")
m.recommend_from_interactions([0])

('Applying transform:\n', Class             : AutoVectorizer

Model Fields
------------
Features          : ['data_1', 'data_2']
Excluded Features : ['my_item_id']

Column  Type   Interpretation  Transforms  Output Type
------  -----  --------------  ----------  -----------
data_1  array  vector          None        array      
data_2  array  vector          None        array      

)


Recsys training: model = item_content_recommender

Defaulting to brute force instead of ball tree because there are multiple distance components.


Starting brute force nearest neighbors model training.

Starting pairwise querying.

...

Data prepared in: 0.013882s

Loading user-provided nearest items.

Generating candidate set for working with new users.

Finished training in 0.000942s

Columns:
	my_item_id	int
	score	float
	rank	int

Rows: 3

Data:
+------------+----------------+------+
| my_item_id |     score      | rank |
+------------+----------------+------+
|     3      | 0.707106769085 |  1   |
|     1      |      0.5       |  2   |
|     2      |      0.5       |  3   |
+------------+----------------+------+
[3 rows x 3 columns]

In [19]:
train, test = graphlab.recommender.util.random_split_by_user(rating_sf, 
                                                               'user_id', 'movie_id')

In [20]:
train[train['rating'] > 4]

Columns:
	user_id	int
	movie_id	int
	rating	int
	timestamp	int

Rows: Unknown

Data:
+---------+----------+--------+-----------+
| user_id | movie_id | rating | timestamp |
+---------+----------+--------+-----------+
|    1    |   1193   |   5    | 978300760 |
|    1    |   2355   |   5    | 978824291 |
|    1    |   1287   |   5    | 978302039 |
|    1    |   2804   |   5    | 978300719 |
|    1    |   595    |   5    | 978824268 |
|    1    |   1035   |   5    | 978301753 |
|    1    |   3105   |   5    | 978301713 |
|    1    |   1270   |   5    | 978300055 |
|    1    |   527    |   5    | 978824195 |
|    1    |    48    |   5    | 978824351 |
+---------+----------+--------+-----------+
[? rows x 4 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.

In [21]:
from graphlab import item_similarity_recommender
itemcf = item_similarity_recommender.create(
    train[train['rating'] > 4], 'user_id', 'movie_id')

Recsys training: model = item_similarity

Warning: Ignoring columns rating, timestamp;

To use one of these as a target column, set target = <column_name>

and use a method that allows the use of a target.

Preparing data set.

...

| 116.515ms                           | 100              | 3228            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.153497s

In [22]:
pop = graphlab.popularity_recommender.create(
    train[train['rating'] > 4], 'user_id', 'movie_id')

Recsys training: model = popularity

Warning: Ignoring columns rating, timestamp;

To use one of these as a target column, set target = <column_name>

and use a method that allows the use of a target.

Preparing data set.

Data has 218844 observations with 6011 users and 3228 items.

Data prepared in: 0.21765s

218844 observations to process; with 3228 unique items.

In [23]:
m = graphlab.recommender.create(
    train, 'user_id', 'movie_id', 'rating')

Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 967831 observations with 6040 users and 3702 items.

Data prepared in: 0.95541s

Training ranking_factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 1.50048

Final training RMSE: 0.937071

In [13]:
m

Class                           : RankingFactorizationRecommender

Schema
------
User ID                         : user_id
Item ID                         : movie_id
Target                          : rating
Additional observation features : 1
Number of user side features    : 0
Number of item side features    : 0

Statistics
----------
Number of observations          : 965508
Number of users                 : 6040
Number of items                 : 3706

Training summary
----------------
Training time                   : 36.9965

Model Parameters
----------------
Model class                     : RankingFactorizationRecommender
num_factors                     : 32
binary_target                   : 0
side_data_factorization         : 1
solver                          : auto
nmf                             : 0
max_iterations                  : 25

Regularization Settings
-----------------------
regularization                  : 0.0
regularization_type             : normal
linear_regulariz

In [14]:
m['coefficients']

{'intercept': 3.5821495005738013, 'movie_id': Columns:
 	movie_id	int
 	linear_terms	float
 	factors	array
 
 Rows: 3706
 
 Data:
 +----------+------------------+-------------------------------+
 | movie_id |   linear_terms   |            factors            |
 +----------+------------------+-------------------------------+
 |   1193   |  1.06781125069   | [-0.119829073548, -0.02245... |
 |   661    | -0.0261590108275 | [-0.727257788181, 0.016146... |
 |   914    |  0.324085891247  | [-0.859803378582, 0.056376... |
 |   3408   |  0.565778970718  | [0.334619760513, -0.014206... |
 |   2355   |  0.648248255253  | [-0.248598009348, 0.103843... |
 |   1197   |  1.12024652958   | [-0.100379563868, 0.085359... |
 |   1287   |  0.345532894135  | [-0.247123196721, 0.024613... |
 |   2804   |  0.894821941853  | [-0.272583067417, 0.046351... |
 |   594    |  0.311594575644  | [-0.974369823933, 0.054282... |
 |   919    |  0.97704321146   | [-0.598346889019, 0.085630... |
 +----------+------------

In [25]:
graphlab.recommender.util.compare_models(test[test['rating'] > 4], 
                                    [pop, itemcf, m], 
                                    user_sample=0.5, 
                                    metric='precision_recall')

compare_models: using 466 users to estimate model performance
PROGRESS: Evaluate model M0

Precision and recall summary statistics by cutoff
+--------+-----------------+-----------------+
| cutoff |  mean_precision |   mean_recall   |
+--------+-----------------+-----------------+
|   1    |  0.111587982833 | 0.0210169663883 |
|   2    |  0.105150214592 | 0.0425562055746 |
|   3    |  0.102288984263 | 0.0544524525898 |
|   4    | 0.0992489270386 | 0.0720982969229 |
|   5    | 0.0909871244635 | 0.0813264276131 |
|   6    | 0.0894134477825 | 0.0945639457791 |
|   7    | 0.0843041079093 |  0.100918470675 |
|   8    | 0.0815450643777 |  0.109968226951 |
|   9    | 0.0810681926562 |  0.126194673165 |
|   10   | 0.0791845493562 |  0.136877932933 |
+--------+-----------------+-----------------+
[10 rows x 3 columns]

PROGRESS: Evaluate model M1

Precision and recall summary statistics by cutoff
+--------+----------------+-----------------+
| cutoff | mean_precision |   mean_recall   |
+------

[{'precision_recall_by_user': Columns:
  	user_id	int
  	cutoff	int
  	precision	float
  	recall	float
  	count	int
  
  Rows: 8388
  
  Data:
  +---------+--------+----------------+----------------+-------+
  | user_id | cutoff |   precision    |     recall     | count |
  +---------+--------+----------------+----------------+-------+
  |    11   |   1    |      0.0       |      0.0       |   6   |
  |    11   |   2    |      0.0       |      0.0       |   6   |
  |    11   |   3    |      0.0       |      0.0       |   6   |
  |    11   |   4    |      0.0       |      0.0       |   6   |
  |    11   |   5    |      0.0       |      0.0       |   6   |
  |    11   |   6    |      0.0       |      0.0       |   6   |
  |    11   |   7    |      0.0       |      0.0       |   6   |
  |    11   |   8    |     0.125      | 0.166666666667 |   6   |
  |    11   |   9    | 0.111111111111 | 0.166666666667 |   6   |
  |    11   |   10   |      0.1       | 0.166666666667 |   6   |
  +---------

### Optimizing for ranking

In [26]:
m_rank = graphlab.recommender.ranking_factorization_recommender.create(
    train, 'user_id', 'movie_id', 'rating', 
    unobserved_rating_value=3)

Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 967831 observations with 6040 users and 3702 items.

Data prepared in: 0.983562s

Training ranking_factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.628853

Final training RMSE: 0.723727

In [27]:
results = graphlab.recommender.util.compare_models(
    test[test['rating'] > 4], 
    [pop, itemcf, m, m_rank], 
    user_sample=0.5, 
    metric='precision_recall')

compare_models: using 466 users to estimate model performance
PROGRESS: Evaluate model M0

Precision and recall summary statistics by cutoff
+--------+-----------------+-----------------+
| cutoff |  mean_precision |   mean_recall   |
+--------+-----------------+-----------------+
|   1    |  0.087982832618 | 0.0140248361318 |
|   2    | 0.0965665236052 | 0.0292522655743 |
|   3    | 0.0937052932761 | 0.0365127169855 |
|   4    | 0.0895922746781 |  0.046837188255 |
|   5    | 0.0862660944206 | 0.0596693591506 |
|   6    | 0.0808297567954 | 0.0685112959099 |
|   7    | 0.0787860208461 | 0.0764441666663 |
|   8    | 0.0769849785408 | 0.0846504186142 |
|   9    | 0.0762994754411 | 0.0972750087562 |
|   10   | 0.0744635193133 |  0.107658443171 |
+--------+-----------------+-----------------+
[10 rows x 3 columns]

PROGRESS: Evaluate model M1

Precision and recall summary statistics by cutoff
+--------+----------------+-----------------+
| cutoff | mean_precision |   mean_recall   |
+------

In [28]:
results[3]['precision_recall_overall']

Columns:
	cutoff	int
	precision	float
	recall	float

Rows: 18

Data:
+--------+----------------+-----------------+
| cutoff |   precision    |      recall     |
+--------+----------------+-----------------+
|   1    | 0.145922746781 | 0.0155536026315 |
|   2    | 0.151287553648 |  0.032545498401 |
|   3    | 0.138769670959 | 0.0480767788701 |
|   4    | 0.128755364807 | 0.0602766563169 |
|   5    | 0.124034334764 | 0.0725574969378 |
|   6    | 0.122675250358 | 0.0837751352493 |
|   7    | 0.115879828326 | 0.0916484679064 |
|   8    | 0.113733905579 |  0.104413957145 |
|   9    | 0.111111111111 |  0.116317958073 |
|   10   | 0.107296137339 |  0.125368902557 |
+--------+----------------+-----------------+
[18 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

### Experimenting with side features

In [29]:
user_sf = graphlab.SFrame('users')
item_sf = graphlab.SFrame('items')

In [30]:
user_sf

Columns:
	user_id	int
	gender	str
	age	str
	occupation	str
	zip-code	str

Rows: 6040

Data:
+---------+--------+-----+------------+----------+
| user_id | gender | age | occupation | zip-code |
+---------+--------+-----+------------+----------+
|    1    |   F    |  1  |     10     |  48067   |
|    2    |   M    |  56 |     16     |  70072   |
|    3    |   M    |  25 |     15     |  55117   |
|    4    |   M    |  45 |     7      |  02460   |
|    5    |   M    |  25 |     20     |  55455   |
|    6    |   F    |  50 |     9      |  55117   |
|    7    |   M    |  35 |     1      |  06810   |
|    8    |   M    |  25 |     12     |  11413   |
|    9    |   M    |  25 |     17     |  61614   |
|    10   |   F    |  35 |     1      |  95370   |
+---------+--------+-----+------------+----------+
[6040 rows x 5 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [31]:
item_sf

Columns:
	movie_id	int
	title	str
	genre	str

Rows: 3883

Data:
+----------+-------------------------------+------------------------------+
| movie_id |             title             |            genre             |
+----------+-------------------------------+------------------------------+
|    1     |        Toy Story (1995)       | Animation|Children's|Comedy  |
|    2     |         Jumanji (1995)        | Adventure|Children's|Fantasy |
|    3     |    Grumpier Old Men (1995)    |        Comedy|Romance        |
|    4     |    Waiting to Exhale (1995)   |         Comedy|Drama         |
|    5     | Father of the Bride Part I... |            Comedy            |
|    6     |          Heat (1995)          |    Action|Crime|Thriller     |
|    7     |         Sabrina (1995)        |        Comedy|Romance        |
|    8     |      Tom and Huck (1995)      |     Adventure|Children's     |
|    9     |      Sudden Death (1995)      |            Action            |
|    10    |        GoldenEye (1995)       |  Action|Adventure|Thriller   |
+----------+-------------------------------+------------------------------+
[3883 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [32]:
m_user = graphlab.recommender.create(train, 'user_id', 'movie_id', 'rating', 
                                     user_data=user_sf)
m_item = graphlab.recommender.create(train, 'user_id', 'movie_id', 'rating', 
                                     item_data=item_sf)
m_both = graphlab.recommender.create(train, 'user_id', 'movie_id', 'rating', 
                                     user_data=user_sf, item_data=item_sf)

Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 967831 observations with 6040 users and 3702 items.

Data prepared in: 0.937442s

Training ranking_factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 1.37294

Final training RMSE: 0.901067

In [33]:
m_both

Class                            : RankingFactorizationRecommender

Schema
------
User ID                          : user_id
Item ID                          : movie_id
Target                           : rating
Additional observation features  : 1
User side features               : ['user_id', 'gender', 'age', 'occupation', 'zip-code']
Item side features               : ['movie_id', 'title', 'genre']

Statistics
----------
Number of observations           : 967831
Number of users                  : 6040
Number of items                  : 3883

Training summary
----------------
Training time                    : 66.8385

Model Parameters
----------------
Model class                      : RankingFactorizationRecommender
num_factors                      : 32
binary_target                    : 0
side_data_factorization          : 1
solver                           : auto
nmf                              : 0
max_iterations                   : 25

Regularization Settings
-------------------

In [35]:
results = graphlab.recommender.util.compare_models(test, 
                                                   [m, m_user, m_item, m_both], 
                                                   user_sample=0.5)

compare_models: using 500 users to estimate model performance
PROGRESS: Evaluate model M0

Precision and recall summary statistics by cutoff
+--------+----------------+-----------------+
| cutoff | mean_precision |   mean_recall   |
+--------+----------------+-----------------+
|   1    |     0.316      | 0.0128883210584 |
|   2    |      0.29      | 0.0233054227061 |
|   3    | 0.268666666667 | 0.0321496282686 |
|   4    |     0.253      | 0.0402442767867 |
|   5    |     0.244      | 0.0490785986855 |
|   6    | 0.232666666667 |  0.055136703757 |
|   7    | 0.225142857143 | 0.0629177137344 |
|   8    |     0.2175     | 0.0678235427485 |
|   9    | 0.211777777778 | 0.0739998255263 |
|   10   |     0.2088     | 0.0813878993056 |
+--------+----------------+-----------------+
[10 rows x 3 columns]

('\nOverall RMSE: ', 0.9912235912964674)

Per User RMSE (best)
+---------+-------+----------------+
| user_id | count |      rmse      |
+---------+-------+----------------+
|   4202  |   2   

In [36]:
[results[i]['rmse_overall'] for i in range(len(results))]

[0.9912235912964674,
 1.0467144960152537,
 1.0571870174756082,
 0.9864932997213498]

In [38]:
results[3]['rmse_by_item']

Columns:
	movie_id	int
	count	int
	rmse	float

Rows: 2552

Data:
+----------+-------+----------------+
| movie_id | count |      rmse      |
+----------+-------+----------------+
|   2871   |   18  | 1.08553449768  |
|   2043   |   3   | 0.28091888274  |
|   2464   |   1   | 0.600349059629 |
|   232    |   5   | 1.99505541159  |
|   3880   |   2   | 1.81932941568  |
|   2238   |   2   | 0.488967829715 |
|   3719   |   4   | 1.49264680177  |
|   431    |   5   | 0.934952171152 |
|   2661   |   2   | 0.782811477717 |
|   3811   |   7   | 0.938744314991 |
+----------+-------+----------------+
[2552 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [39]:
graphlab.recommender.util.compare_models(test[test['rating'] > 4], 
                                    [m_rank, m_both], 
                                    user_sample=0.2, 
                                    metric='precision_recall')

compare_models: using 186 users to estimate model performance
PROGRESS: Evaluate model M0

Precision and recall summary statistics by cutoff
+--------+-----------------+-----------------+
| cutoff |  mean_precision |   mean_recall   |
+--------+-----------------+-----------------+
|   1    |  0.102150537634 | 0.0103584423342 |
|   2    |  0.134408602151 | 0.0325592569006 |
|   3    |  0.123655913978 | 0.0469152266463 |
|   4    |  0.103494623656 | 0.0536329312206 |
|   5    | 0.0978494623656 | 0.0668332524048 |
|   6    | 0.0905017921147 | 0.0706819521975 |
|   7    | 0.0867895545315 | 0.0798389988545 |
|   8    | 0.0880376344086 | 0.0929486544744 |
|   9    |  0.089605734767 |  0.108265520447 |
|   10   | 0.0876344086022 |  0.117956009872 |
+--------+-----------------+-----------------+
[10 rows x 3 columns]

PROGRESS: Evaluate model M1

Precision and recall summary statistics by cutoff
+--------+----------------+-----------------+
| cutoff | mean_precision |   mean_recall   |
+------

[{'precision_recall_by_user': Columns:
  	user_id	int
  	cutoff	int
  	precision	float
  	recall	float
  	count	int
  
  Rows: 3348
  
  Data:
  +---------+--------+----------------+----------------+-------+
  | user_id | cutoff |   precision    |     recall     | count |
  +---------+--------+----------------+----------------+-------+
  |    56   |   1    |      1.0       | 0.166666666667 |   6   |
  |    56   |   2    |      0.5       | 0.166666666667 |   6   |
  |    56   |   3    | 0.333333333333 | 0.166666666667 |   6   |
  |    56   |   4    |      0.25      | 0.166666666667 |   6   |
  |    56   |   5    |      0.4       | 0.333333333333 |   6   |
  |    56   |   6    | 0.333333333333 | 0.333333333333 |   6   |
  |    56   |   7    | 0.285714285714 | 0.333333333333 |   6   |
  |    56   |   8    |      0.25      | 0.333333333333 |   6   |
  |    56   |   9    | 0.222222222222 | 0.333333333333 |   6   |
  |    56   |   10   |      0.2       | 0.333333333333 |   6   |
  +---------

### Factorization machines

In [ ]:
fm = graphlab.recommender.create(train.head(10000), 'user_id', 'movie_id', 'rating',
                                 method='factorization_model',
                                 item_data=item_sf, 
                                 sgd_step_size=0.09,
                                 max_iterations=10)